### Data Collection


1. GET S&P 500 company info<br>


In [13]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [14]:
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from newsapi import NewsApiClient
from pathlib import Path
%matplotlib inline

[nltk_data] Downloading package vader_lexicon to C:\Users\Straw
[nltk_data]     Hat\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [15]:
from bs4 import BeautifulSoup
import datetime
import unicodedata
import requests
import pandas as pd
import numpy as np
from time import sleep
import re

class SEC_Extractor:
    def get_doc_links(cik,ticker):
        try:
            base_url = "https://www.sec.gov/cgi-bin/browse-edgar"
            inputted_cik = cik
            payload = {
                "action" : "getcompany",
                "CIK" : inputted_cik,
                "type" : "8-K",
                "output":"xml",
                "dateb" : "20180401",
            }
            sec_response = requests.get(url=base_url,params=payload)
            soup = BeautifulSoup(sec_response.text,'lxml')
            url_list = soup.findAll('filinghref')
            html_list = []
            # Get html version of links
            for link in url_list:
                link = link.string
                if link.split(".")[len(link.split("."))-1] == 'htm':
                    txtlink = link + "l"
                    html_list.append(txtlink)

            doc_list = []
            doc_name_list = []
            # Get links for txt versions of files
            for k in range(len(html_list)):
                txt_doc = html_list[k].replace("-index.html",".txt")
                doc_name = txt_doc.split("/")[-1]
                doc_list.append(txt_doc)
                doc_name_list.append(doc_name)
                # Create dataframe of CIK, doc name, and txt link
            df = pd.DataFrame(
                {
                "cik" : [cik]*len(html_list),
                "ticker" : [ticker]*len(html_list),
                "txt_link" : doc_list,
                "doc_name": doc_name_list
                }
            )
        except requests.exceptions.ConnectionError:
                sleep(.1)
        return df

    # Extracts text and submission datetime from document link
    def extract_text(link):
        try:
            r = requests.get(link)
            #Parse 8-K document
            filing = BeautifulSoup(r.content,"html5lib",from_encoding="ascii")
            #Extract datetime
            try:
                submission_dt = filing.find("acceptance-datetime").string[:14]
            except AttributeError:
                    # Flag docs with missing data as May 1 2018 10AM
                submission_dt = "20180501100000"
            
            submission_dt = datetime.datetime.strptime(submission_dt,"%Y%m%d%H%M%S")
            #Extract HTML sections
            for section in filing.findAll("html"):
                try:
                    #Remove tables
                    for table in section("table"):
                        table.decompose()
                    #Convert to unicode
                    section = unicodedata.normalize("NFKD",section.text)
                    section = section.replace("\t"," ").replace("\n"," ").replace("/s"," ").replace("\'","'")            
                except AttributeError:
                    section = str(section.encode('utf-8'))
            filing = "".join((section))
        except requests.exceptions.ConnectionError:
                sleep(10)
        sleep(.1)

        return filing, submission_dt

    def extract_item_no(document):
        pattern = re.compile("Item+ +\d+[\:,\.]+\d+\d")
        item_list = re.findall(pattern,document)
        return item_list


### 1. Get S&P 500 Data

In [16]:
# Get table of the S&P 500 tickers, CIK, and industry from Wikipedia
wiki_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
cik_df = pd.read_html(wiki_url,header=0,index_col=0)[0]
cik_df['GICS Sector'] = cik_df['GICS Sector'].astype("category")
cik_df['GICS Sub Industry'] = cik_df['GICS Sector'].astype("category")
cik_df.head()

,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded,GICS Sub Industry
Symbol,,,,,,,,,
MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902,Industrials
AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916,Industrials
ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888,Health Care
ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888),Health Care
ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981,Health Care


In [19]:
from newsapi import NewsApiClient
load_dotenv()
newsapi = NewsApiClient(api_key=os.environ["NEWS_API_KEY"])

In [20]:
headlines = newsapi.get_everything(
    q="S&P 500",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

In [22]:
# Create the S&p 500 sentiment scores DataFrame
sentiments = []

for article in headlines["articles"]:
    try:
        
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        sentiments.append({
            
            
            "Compound": compound,
            "Positive": pos,
            "Negative": neg,
            "Neutral": neu,
            "Text": text
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
df = pd.DataFrame(sentiments)

# Reorder DataFrame columns
cols = ["Compound", "Positive", "Negative", "Neutral","Text"]
df = df[cols]

df.head(10)

,Compound,Positive,Negative,Neutral,Text
0,0.6369,0.148,0.000,0.852,Feb 11 - Welcome to the home for real-time cov...
1,-0.6249,0.000,0.124,0.876,Posted \r\nWall Street's main indexes fell on ...
2,0.1531,0.046,0.000,0.954,There are plenty of solid reasons why investor...
3,0.6369,0.148,0.000,0.852,Feb 9 - Welcome to the home for real-time cove...
4,0.6908,0.151,0.000,0.849,Feb 10 (Reuters) - Futures tracking the S&amp;...
5,0.6369,0.144,0.000,0.856,Feb 11 - Welcome to the home for real-time cov...
6,-0.4404,0.000,0.086,0.914,NEW YORK (Reuters) - The S&amp;P 500 index end...
7,0.6369,0.152,0.000,0.848,Feb 7 - Welcome to the home for real-time cove...
8,-0.5859,0.000,0.127,0.873,Feb 14 (Reuters) - Goldman Sachs has tempered ...
9,0.0000,0.000,0.000,1.000,"NEW YORK, Feb 22 (Reuters) - The S&amp;P 500's..."


In [23]:
df.describe()

,Compound,Positive,Negative,Neutral
count,100.000000,100.000000,100.000000,100.000000
mean,0.168539,0.101600,0.054140,0.844230
std,0.532687,0.081072,0.069649,0.082521
min,-0.836000,0.000000,0.000000,0.633000
25%,-0.307000,0.000000,0.000000,0.791250
50%,0.261600,0.118000,0.000000,0.849500
75%,0.636900,0.152000,0.111250,0.873750
max,0.910000,0.287000,0.282000,1.000000


In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Expand the default stopwords list if necessary
sw = set(stopwords.words('english'))